In [4]:
import pandas as pd 
import numpy as np 
import gurobipy as gp 

## Question 1

In [5]:
path = {('s', 'a'):1, ('s', 'b'):4, ('a', 'b'):2 ,('a', 'c'):6, ('b', 'd'):1, 
        ('b', 'e'):2, ('c', 'f'):2, ('d', 'c'):3,('d', 'g'):7, ('e', 'd'):3, ('f', 'g'):1}

In [15]:
m = gp.Model('q1')

alloc = {i: m.addVar(vtype=gp.GRB.BINARY, name=f'{i}') for i in path}

# minimize the shorted distance for a given path assignment
m.setObjective(sum(alloc[i] * path[i] for i in alloc), gp.GRB.MINIMIZE)

# constraints to flow
m.addConstr(alloc['s', 'a'] + alloc['s', 'b'] == 1) # start
m.addConstr(alloc['f', 'g'] + alloc['d', 'g'] == 1) # end

m.addConstr(alloc['s', 'a'] == alloc['a', 'c'] + alloc['a', 'b'])
m.addConstr(alloc['s', 'b'] + alloc['a', 'b'] == alloc['b', 'd'] + alloc['b', 'e'])
m.addConstr(alloc['a', 'c'] + alloc['d', 'c'] == alloc['c', 'f'])
m.addConstr(alloc['b', 'd'] + alloc['e', 'd'] == alloc['d', 'c'] + alloc['d', 'g'])
m.addConstr(alloc['b', 'e'] == alloc['e', 'd'])
m.addConstr(alloc['c', 'f'] == alloc['f', 'g'])

m.optimize()

if m.status == gp.GRB.OPTIMAL:
    for i in alloc:
        if alloc[i].x == 1:
            print(f'path {i} = {alloc[i].x}')



Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8 rows, 11 columns and 22 nonzeros
Model fingerprint: 0x06faff3e
Variable types: 0 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 8 rows and 11 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%
path ('s', 'a') = 1.0
path ('a', 'c') = 1.0
path (

## Question 2

In [20]:
workers = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
machine = ['m1', 'm2', 'm3', 'm4', 'm5']

premium = {
    'a': {'m1': 36, 'm2': 34, 'm3': 32, 'm4': 35, 'm5': 36},
    'b': {'m1': 21, 'm2': 25, 'm3': 24, 'm4': 25, 'm5': 26},
    'c': {'m1': 21, 'm2': 24, 'm3': 24, 'm4': 25, 'm5': 24},
    'd': {'m1': 34, 'm2': 33, 'm3': 37, 'm4': 28, 'm5': 37},
    'e': {'m1': 29, 'm2': 24, 'm3': 26, 'm4': 30, 'm5': 29},
    'f': {'m1': 31, 'm2': 30, 'm3': 30, 'm4': 32, 'm5': 31},
    'g': {'m1': 27, 'm2': 25, 'm3': 26, 'm4': 27, 'm5': 29}
          }

In [21]:
pd.DataFrame(premium)

,a,b,c,d,e,f,g
m1,36,21,21,34,29,31,27
m2,34,25,24,33,24,30,25
m3,32,24,24,37,26,30,26
m4,35,25,25,28,30,32,27
m5,36,26,24,37,29,31,29


In [24]:
m = gp.Model('q2')

alloc = m.addVars(workers, machine, vtype=gp.GRB.BINARY)

# minimize the insurance premium for a given worker assignment
m.setObjective(sum(premium[i][j] * alloc[i, j] for i in workers for j in machine), gp.GRB.MINIMIZE)

# worker constraints:every worker has to be matched to a single machine 
for i in workers:
    m.addConstr(sum(alloc[i, j] for j in machine) == 1)
    
# machine constraints:every machine must have at least one person working on it 
for j in machine:
    m.addConstr(sum(alloc[i, j] for i in workers) >= 1)

m.optimize()

if m.status == gp.GRB.OPTIMAL:
    for i in alloc:
        if alloc[i].x == 1:
            print(f'path {i} = {alloc[i].x}')


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 12 rows, 35 columns and 70 nonzeros
Model fingerprint: 0xa0d6bf18
Variable types: 0 continuous, 35 integer (35 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 204.0000000
Presolve time: 0.00s
Presolved: 12 rows, 35 columns, 70 nonzeros
Variable types: 0 continuous, 35 integer (35 binary)
Found heuristic solution: objective 192.0000000

Root relaxation: objective 1.820000e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     

## Question 3

In [57]:
terminals = {1:'SE', 2:'LA', 3:'DE', 4:'CH', 5:'DA', 6:'NY', 7:'DC'}
flip_t = {'SE':1, 'LA':2, 'DE':3, 'CH':4, 'DA':5, 'NY':6, 'DC':7}

# hard assign the value without the LA -> CH terminal i.e. 2 -> 4 (we instantiate the MST with this edge)
distance = {
    ('SE', 'DE'):1300, ('SE', 'LA'):1100, ('SE', 'CH'):2000, ('LA', 'DA'):1400, 
    ('LA', 'DC'):2600, ('DE', 'DA'):780, ('DE', 'CH'):1000, 
    ('DA', 'CH'):900, ('DA', 'DC'):1300, ('CH', 'NY'):800, ('DC', 'NY'):200
}

useful_distance = {(flip_t[i[0]], flip_t[i[1]]) : distance[i] for i in distance}

In [58]:
# sorting edge based on weight 
sorted_edges = sorted(useful_distance.items(), key=lambda x: x[1])

In [59]:
sorted_edges

[((7, 6), 200),
 ((3, 5), 780),
 ((4, 6), 800),
 ((5, 4), 900),
 ((3, 4), 1000),
 ((1, 2), 1100),
 ((1, 3), 1300),
 ((5, 7), 1300),
 ((2, 5), 1400),
 ((1, 4), 2000),
 ((2, 7), 2600)]

In [60]:
# instantiate the parent class ID
parent = {i: i for i in range(1,7+1)}
rank = {i: 0 for i in range(1,7+1)}


# HELPER FUNCTIONS
def find(node):
    if parent[node] != node:
        parent[node] = find(parent[node])
    return parent[node]

def union(node1, node2):
    root1 = find(node1)
    root2 = find(node2)
    
    if root1 != root2:
        if rank[root1] > rank[root2]:
            parent[root2] = root1
        else:
            parent[root1] = root2
            if rank[root1] == rank[root2]:
                rank[root2] += 1
                
mst = []

new_list = [((2,4), 2000)] + sorted_edges
# Kruskal's Algo
for edge, weight in new_list:
    if find(edge[0]) != find(edge[1]):
        union(edge[0], edge[1])
        mst.append(edge)
    
# output edge path
total_distance = 2000
for edge in mst:
    print(f'{edge} -> {terminals[edge[0]]},{terminals[edge[1]]}')
    
    if edge != (2,4):
        total_distance += useful_distance[edge]
    
print(f'Total Distance = {total_distance}')

(2, 4) -> LA,CH
(7, 6) -> DC,NY
(3, 5) -> DE,DA
(4, 6) -> CH,NY
(5, 4) -> DA,CH
(1, 2) -> SE,LA
Total Distance = 5780


In [65]:
# Part 3 of Question 3

In [64]:
terminals = {1:'SE', 2:'LA', 3:'DE', 4:'CH', 5:'DA', 6:'NY', 7:'DC', 8:'VA', 9:'MT', 10:'TR'}
flip_t = {'SE':1, 'LA':2, 'DE':3, 'CH':4, 'DA':5, 'NY':6, 'DC':7, 'VA':8, 'MT':9, 'TR':10}

# hard assign the value without the LA -> CH terminal i.e. 2 -> 4 (we instantiate the MST with this edge)
distance = {
    ('SE', 'DE'):1300, ('SE', 'LA'):1100, ('SE', 'CH'):2000, ('LA', 'DA'):1400, 
    ('LA', 'DC'):2600, ('DE', 'DA'):780, ('DE', 'CH'):1000, 
    ('DA', 'CH'):900, ('DA', 'DC'):1300, ('CH', 'NY'):800, ('DC', 'NY'):200,
    
    ('SE', 'VA'):700, ('DE', 'VA'):600, ('CH', 'MT'):900, 
    ('NY', 'MT'):500, ('NY', 'TR'):600, ('DC', 'TR'):700, ('CH', 'TR'):600
}

useful_distance = {(flip_t[i[0]], flip_t[i[1]]) : distance[i] for i in distance}

# sorting edge based on weight 
sorted_edges = sorted(useful_distance.items(), key=lambda x: x[1])

# instantiate the parent class ID
parent = {i: i for i in range(1,10+1)}
rank = {i: 0 for i in range(1,10+1)}

# HELPER FUNCTIONS
def find(node):
    if parent[node] != node:
        parent[node] = find(parent[node])
    return parent[node]

def union(node1, node2):
    root1 = find(node1)
    root2 = find(node2)
    
    if root1 != root2:
        if rank[root1] > rank[root2]:
            parent[root2] = root1
        else:
            parent[root1] = root2
            if rank[root1] == rank[root2]:
                rank[root2] += 1
                
mst = []

new_list = [((2,4), 2000)] + sorted_edges

# Kruskal's Algo
for edge, weight in new_list:
    if find(edge[0]) != find(edge[1]):
        union(edge[0], edge[1])
        mst.append(edge)
    
# output edge path
total_distance = 2000
for edge in mst:
    print(f'{edge} -> {terminals[edge[0]]},{terminals[edge[1]]}')
    
    if edge != (2,4):
        total_distance += useful_distance[edge]
    
print(f'Total Distance = {total_distance}')

(2, 4) -> LA,CH
(7, 6) -> DC,NY
(6, 9) -> NY,MT
(3, 8) -> DE,VA
(6, 10) -> NY,TR
(4, 10) -> CH,TR
(1, 8) -> SE,VA
(3, 5) -> DE,DA
(5, 4) -> DA,CH
Total Distance = 6880
